In [ ]:
### Run create_dataset.ipynb first. Ensure you have the train_df saved in {train_df_path}

### Select training data that are of label 1

In [ ]:
from load_dataset import separate_labels

df_label_0, df_label_1 = separate_labels(file_path=) # Enter the train_df_path
# Remember we should generate label 1 data
# Make sure that df_label_0, df_label_1 are consistent
real_data = df_label_1

### Train TVAE Model

In [ ]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata(df_label_1)
metadata.detect_from_dataframe(df_label_1) # TODO: Fix
metadata

In [ ]:
from sdv.single_table import TVAESynthesizer
synthesizer = TVAESynthesizer(metadata, verbose = True)
synthesizer.fit(df_label_1)

In [ ]:
synthesizer.save('tvae_synthesizer.pkl')

### Sample Data

In [ ]:
# If have the synthesizer saved just load it
# import joblib
# synthesizer = joblib.load("tvae_synthesizer.pkl")

In [ ]:
# How many synthetic data we need?
num_generated_data_needed = df_label_0.shape[0] - df_label_1.shape[0]
synthetic_data = synthesizer.sample(num_generated_data_needed)
synthetic_data.to_csv("{Path to save the synthetic data}")

### Evaluation

In [ ]:
from load_dataset import load_dataset
# If have the synthetic data saved just load it, however if you have the synthetic data generated in the previous step you can ignore this
synthetic_data = load_dataset(file_path=) # Enter path to the synthetic data

Evaluation: Fidelity

In [ ]:
from evaluation.fidelity import *
# PCA
plot_pca(real_data=real_data, synthetic_data=synthetic_data)

In [ ]:
# ks_test
ks_test(real_data=real_data, synthetic_data=synthetic_data)

In [ ]:
# KDE Plot
kde_plot(real_data=real_data, synthetic_data=synthetic_data)

Evaluation: Utility

In [ ]:
# Train dataset + test datase
test_df = load_dataset(file_path=) # Enter path to the test dataset

# Combine synthetic data and real data for the train set
combined_train_data = pd.concat([real_data, synthetic_data], ignore_index=True)
X_train = combined_train_data.drop(columns=["label"])
y_train = combined_train_data["label"]
X_test = test_df.drop(columns=["label"])
y_test = test_df["label"]

In [ ]:
from evaluation.utility import *
# Logistic Regression
evaluate(method="logistic_regression", X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
# Random Forest
evaluate(method="random_forest", X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
# Catboost
evaluate(method="catboost", X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

Evaluation: Privacy (TODO)